In [1]:
import numpy as np
import cv2 as ocv
import scipy.stats as scs
import matplotlib.pyplot as plt
import cmath

In [19]:
def countHomog(b):
    nb = len(b)
    y = 0
    for i in range(1, nb):
        y += b[i] * b [i - 1] + (1 - b[i]) * (1 - b[i - 1])

    return y / nb

In [20]:
img_r = ocv.imread("./pup.jpg", ocv.IMREAD_GRAYSCALE)
img = img_r.flatten()

# LSB flip 

vFlip = []

for i in img:
    if i % 2 == 0:
        vFlip.append(i-1)
    else:
        vFlip.append(i+1)

        

In [21]:
def ColorCut(a, x, y, z):
    for i in range(0, len(x)):
        if(x[i] == y or x[i] == z):
            a.append(x[i] % 2)

In [22]:
def quadratic_solution(a, b, c):    
    d = (b**2) - (4 * a * c)
    sol1 = ((-b - cmath.sqrt(d)) / (2*a))
    sol2 = ((-b + cmath.sqrt(d)) / (2*a))

    return sol2.real, sol1.real        

In [23]:
def calcR_half(Zd):
    result = 0
    p = 2
    
    print(len(Zd))
    for k in range(1, len(Zd)):
        hk = 0

        # pragma omp for shared(hk, result) num_threads(8) reduction(+:hk) 
        for j in range(0, len(Zd)-k):
            hk += countHomog(Zd[j:j+k])
        
        result += hk/p
        p *= 2

        print(k)
    return result
            

In [24]:
Z = []
Zd = []
Zdd = []


for k in range(0,128):
    ColorCut(Z, img, 2*k, 2*k+1)

for k in range(0, 127):
    ColorCut(Zd, img, 2*k + 1, 2*k + 2)
    ColorCut(Zdd, vFlip, 2*k + 1, 2*k + 2)

ColorCut(Zd, img, 255, 1)
ColorCut(Zdd, vFlip, 255, 1)

In [25]:
D_Beta = countHomog(Z) - countHomog(Zd)
D_1_Beta = countHomog(Z) - countHomog(Zdd)
D_half = calcR_half(Zd)

a = 4*D_half 
b = D_1_Beta - D_Beta - a 
c = D_Beta

betaP, betaM = quadratic_solution(a,b,c)

beta =  min(betaP,betaM)

print(beta)





KeyboardInterrupt: 